In [92]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

import os
import json

In [93]:
def add_multicolumn(df, df2, new_col_name):
    tmp = df2.copy()    # make copy, otherwise df2 will be changed !!!
    tmp.columns = pd.MultiIndex.from_product([[new_col_name], df2.columns.tolist()])
    return pd.concat([df, tmp], axis=1)

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [122]:
model_path = '/users/Stathis/GoogleDrive/MLP/attack_results_pgd_7/'
all_nets =  list(os.listdir(model_path))

# Label Leaking

In [134]:
nets_original_baselines = [a for a in all_nets if 'pgd_7' in a and 'no_LL' not in a and 'all_layers' not in a and 'transfer' not in a
                        and 'cifar100' not in a]
nets_original_baselines.sort()
nets_original_baselines

['white_box_attacks_pgd_7_densenet121_cifar10.json',
 'white_box_attacks_pgd_7_densenet121_cifar10_fgsm.json',
 'white_box_attacks_pgd_7_densenet121_cifar10_pgd.json',
 'white_box_attacks_pgd_7_resnet56_cifar10.json',
 'white_box_attacks_pgd_7_resnet56_cifar10_fgsm.json',
 'white_box_attacks_pgd_7_resnet56_cifar10_pgd.json']

In [130]:
nets_original_transfer = [a for a in all_nets if 'pgd_7' in a and 'no_LL' not in a and 'all_layers' not in a and 'transfer' in a]
nets_original_transfer.sort()

In [131]:
nets_original_transfer

['white_box_attacks_pgd_7_transfer_densenet121_fgsm_fgsm.json',
 'white_box_attacks_pgd_7_transfer_densenet121_fgsm_nat.json',
 'white_box_attacks_pgd_7_transfer_densenet121_nat_fgsm.json',
 'white_box_attacks_pgd_7_transfer_densenet121_nat_nat.json',
 'white_box_attacks_pgd_7_transfer_densenet121_nat_pgd.json',
 'white_box_attacks_pgd_7_transfer_densenet121_pgd_nat.json',
 'white_box_attacks_pgd_7_transfer_densenet121_pgd_pgd.json',
 'white_box_attacks_pgd_7_transfer_resnet56_fgsm_fgsm.json',
 'white_box_attacks_pgd_7_transfer_resnet56_fgsm_nat.json',
 'white_box_attacks_pgd_7_transfer_resnet56_nat_fgsm.json',
 'white_box_attacks_pgd_7_transfer_resnet56_nat_nat.json',
 'white_box_attacks_pgd_7_transfer_resnet56_nat_pgd.json',
 'white_box_attacks_pgd_7_transfer_resnet56_pgd_nat.json',
 'white_box_attacks_pgd_7_transfer_resnet56_pgd_pgd.json']

In [133]:
nets = nets_original_transfer

attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[net[24:-5]] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM_0.03125', 'PGD_0.03125',
                                                     'FGSM_0.0625', 'PGD_0.0625',
                                                     'FGSM_0.125', 'PGD_0.125',                                                    
                                                    ], orient='index')
df=np.round(df*100,1)

for net in df.iterrows():
#     print(net[0])
    nn = net[0].split('_')
#     print(nn)
    df.loc[net[0],'net'] = nn[3]
    df.loc[net[0],'source'] = nn[4]
    df.loc[net[0],'target'] = nn[5]
    df.loc[net[0],'step'] = nn[7]
    df.loc[net[0],'gamma'] = nn[9]
    
df.groupby([ 'net','source', 'target', ]).mean()

AssertionError: 7 columns passed, passed data had 3 columns

# No Label Leaking

In [90]:
model_path = '/users/Stathis/GoogleDrive/MLP/attack_results_pgd_7/'
nets =  list(os.listdir(model_path))
nets = [a for a in nets if 'all_layers' in a and 'no_LL' in a]
nets.sort()

attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[net[30:-5]] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM_0.03125', 'PGD_0.03125',
                                                     'FGSM_0.0625', 'PGD_0.0625',
                                                     'FGSM_0.125', 'PGD_0.125',                                                    
                                                    ], orient='index')
df=np.round(df*100,1)

for net in df.iterrows():
#     print(net[0])
    nn = net[0].split('_')
#     print(nn)
    df.loc[net[0],'net'] = nn[3]
    df.loc[net[0],'source'] = nn[4]
    df.loc[net[0],'target'] = nn[5]
    df.loc[net[0],'step'] = nn[7]
    df.loc[net[0],'gamma'] = nn[9]
    
# df.set_index([ 'net','source', 'target', ])

In [91]:
df.groupby([ 'net','source', 'target', ]).mean()

Clean  FGSM_0.03125  PGD_0.03125  FGSM_0.0625  \
net         source target                                                  
densenet121 fgsm   fgsm    91.50         74.70        56.10        64.60   
                   nat     93.95         58.00        16.80        46.25   
            nat    fgsm    90.40         76.00        60.60        65.80   
                   nat     93.45         53.50        11.60        42.30   
                   pgd     89.40         72.40        55.80        62.10   
            pgd    nat     92.70         57.00        17.70        45.50   
                   pgd     89.40         74.80        55.30        63.20   
resnet56    fgsm   fgsm    89.20         68.55        49.35        54.35   
                   nat     91.85         56.65        18.15        44.80   
            nat    fgsm    89.90         73.60        61.65        61.50   
                   nat     92.75         55.40        13.15        43.30   
                   pgd     88.10         73.80        60.30        63.70   
            pgd    nat     91.55         58.00        16.45        47.35   
                   pgd     88.70         72.80        54.90        62.20   

                           PGD_0.0625  FGSM_0.125  PGD_0.125  
net         source target                                     
densenet121 fgsm   fgsm         31.30       49.70       8.80  
                   nat           3.70       35.60       0.65  
            nat    fgsm         36.50       48.60      11.70  
                   nat           2.50       32.55       0.45  
                   pgd          30.20       47.60       8.00  
            pgd    nat           4.15       34.45       0.80  
                   pgd          29.80       47.00       7.60  
resnet56    fgsm   fgsm         23.20       38.25       5.75  
                   nat           4.50       34.40       0.80  
            nat    fgsm         38.50       43.60      13.35  
                   nat           2.75       33.25       0.55  
                   pgd          36.70       47.20      12.40  
            pgd    nat           4.40       37.05       0.75  
                   pgd          29.90       46.60       8.90

# Black box attacks

In [74]:
js = json.load(open("../experiments_results/attack_results/black_box_results_all_layers.json"))

In [83]:
# js